In [70]:
import pandas as pd
import numpy as np
import os
from typing import List

##### Load and merge datasets we need:

In [71]:
def load_stata_files(data_directory):
    file_names = [
        'sharew2_rel8-0-0_ph.dta',
        'sharew2_rel8-0-0_br.dta',
        'sharew2_rel8-0-0_ws.dta',
        'sharew2_rel8-0-0_cs.dta',
        'sharew2_rel8-0-0_gs.dta',
        'sharew2_rel8-0-0_dn.dta',
        'sharew2_rel8-0-0_gv_isced.dta'
    ]

    datasets = {}
    for file_name in file_names:
        file_path = os.path.join(data_directory, file_name)
        dataset_name = os.path.splitext(file_name)[0]
        datasets[dataset_name] = pd.read_stata(file_path)
    
    return datasets

def get_common_columns(df_primary, df_secondary, merge_keys=['mergeid']):
    unique_cols = df_secondary.columns.difference(df_primary.columns).tolist()
    return unique_cols + merge_keys

def merge_datasets(datasets, merge_key='mergeid'):
    main_key = 'sharew2_rel8-0-0_ph'
    df_merged = datasets[main_key].copy()
    
    for key in datasets:
        if key == main_key:
            continue
        cols_to_use = get_common_columns(df_merged, datasets[key], merge_keys=[merge_key])
        df_merged = df_merged.merge(
            datasets[key][cols_to_use],
            on=merge_key,
            how='left'
        )
    return df_merged

In [72]:
data_dir = './data/wave2/'
datasets = load_stata_files(data_dir)
df = merge_datasets(datasets)

In [73]:
df

,mergeid,hhid2,mergeidp2,coupleid2,country,language,ph003_,ph004_,ph005_,ph006d1,...,dn041_,dn042_,dn043_,dn044_,isced1997_c1,isced1997_c2,isced1997_c3,isced1997_c4,isced1997_r,isced1997_sp
0,AT-000327-01,AT-000327-A,AT-000327-02,AT-000327-01-02,Austria,German (at),Poor,Yes,Severely limited,Not selected,...,Implausible/suspected wrong,Male,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN
1,AT-000327-02,AT-000327-A,AT-000327-01,AT-000327-01-02,Austria,German (at),Very good,No,Not limited,Not selected,...,3.0,Female,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 2,ISCED-97 code 3,NaN
2,AT-001816-02,AT-001816-A,AT-001816-01,AT-001816-01-02,Austria,German (at),Good,No,Not limited,Not selected,...,8.0,Male,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 6,NaN,NaN,ISCED-97 code 3,NaN
3,AT-002132-01,AT-002132-A,,,Austria,German (at),Poor,Yes,Severely limited,Not selected,...,10.0,Female,Yes,"No, marital status has not changed",ISCED-97 code 4,NaN,NaN,NaN,ISCED-97 code 2,ISCED-97 code 1
4,AT-004234-01,AT-004234-A,AT-004234-02,AT-004234-01-02,Austria,German (at),Fair,Yes,Severely limited,Not selected,...,8.0,Male,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37138,SE-998278-01,SE-998278-A,SE-998278-02,SE-998278-01-02,Sweden,Swedish,Very good,No,Not limited,Not selected,...,13.0,Female,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,NaN,ISCED-97 code 2,NaN
37139,SE-998807-01,SE-998807-A,SE-998807-02,SE-998807-01-02,Sweden,Swedish,Very good,No,Not limited,Not selected,...,13.0,Female,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 5,NaN,NaN,ISCED-97 code 5,NaN
37140,SE-998807-02,SE-998807-A,SE-998807-01,SE-998807-01-02,Sweden,Swedish,Good,No,"Limited, but not severely",Not selected,...,15.0,Male,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 5,NaN
37141,SE-998962-01,SE-998962-A,,,Sweden,Swedish,Very good,No,Not limited,Selected,...,7.0,Male,Yes,"No, marital status has not changed",ISCED-97 code 4,ISCED-97 code 4,NaN,NaN,ISCED-97 code 1,ISCED-97 code 3


##### Rename variables

In [74]:
vars = {
'ws010_' :  'first_walking_speed_result',
'ws011_' :  'first_walking_speed_time',
'ws012_' :  'second_walking_speed_result',
'ws013_' :  'second_walking_speed_time',
'cs004_' :  'chair_rise_result',
'cs008_' :  'time_5_chair_rise',
'gs006_' :  'maxgrip_left_one',
'gs007_' :  'maxgrip_left_two',
'gs008_' :  'maxgrip_right_one',
'gs009_' :  'maxgrip_right_two',
'dn002_' :  'MOB', #In which month and year were you born? MONTH: YEAR:
'dn003_' :  'YOB', #In which month and year were you born? MONTH: [DN002_MoBirth] YEAR:
'dn042_' :  'female',
'ph006d8' :  'OA',
'ph010d1' :  'pain_joint',
'ph012_' :  'weight',
'ph013_' :  'height',
'ph044_' :  'eyesight_close',
'ph046_' :  'hearing',
}

df.rename(columns=vars, inplace=True)

##### Mapping educational level

In [75]:
def map_education_levels(df, column='isced1997_r'):
    """
    Maps ISCED-97 education codes to labeled categories and then to integer levels.

    Parameters:
    - df (pd.DataFrame): The input DataFrame
    - column (str): The name of the column with ISCED-97 codes

    Returns:
    - df (pd.DataFrame): The same DataFrame with 'educational' and 'educational_level' columns added
    """
    education_map = {
        'ISCED-97 code 1': 'Low',
        'ISCED-97 code 2': 'Low',
        'ISCED-97 code 3': 'Medium',
        'ISCED-97 code 4': 'Medium',
        'ISCED-97 code 5': 'High',
        'ISCED-97 code 6': 'High',
        'None': 'Other',
        'Other': 'Other',
        'Refusal': 'Other',
        'Still in school': 'Other',
        "Don't know": 'Other'
    }

    level_to_int = {
        'Low': 0,
        'Medium': 1,
        'High': 2,
        'Other': 3
    }

    df['educational'] = df[column].map(education_map)
    df['educational_level'] = df['educational'].map(level_to_int)

    return df

df = map_education_levels(df)

##### Mapping Categorical Responses to Numeric Values

In [76]:
df.loc[:, 'female'] = df['female'].replace({
    'Male': 0,
    'Female': 1,
})

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/1272340355.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'female'] = df['female'].replace({
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/1272340355.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df.loc[:, 'female'] = df['female'].replace({
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/1272340355.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0, 1, 

In [77]:
def convert_selected_to_numeric(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Convert categorical 'Selected'/'Not selected' responses in specified columns to numeric format.
    """
    df = df.copy()
    replacement_map = {
        'Selected': 1,
        'Not selected': 0,
        "Don't know": np.nan,
        'Refusal': np.nan,
        'Unable to complete task': np.nan,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(replacement_map)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

columns_to_convert = [
    'ph048dno', 'ph049dno', 'ph006d19', 'ph006d20', 'ph087d2', 'ph087d3', 'ph006d1', 'ph006d4',
    'ph006d16', 'ph006d11', 'ph006d5', 'ph006d21', 'ph006d10', 'ph008d22', 'ph008d21', 'ph006d2',
    'ph006d5', 'second_walking_speed_time', 'first_walking_speed_time', 'time_5_chair_rise',
    'maxgrip_left_one', 'maxgrip_left_two', 'maxgrip_right_one', 'maxgrip_right_two', 'YOB', 'MOB',
    'OA', 'pain_joint', 'weight', 'height'
]

df = convert_selected_to_numeric(df, columns_to_convert)


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/2875467203.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(replacement_map)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/2875467203.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(replacement_map)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/2875467203.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old beha

In [78]:
def map_chair_rise_results(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Maps chair rise results to numeric format.
    """
    df = df.copy()
    mapping = {
        'Respondent stood up without using arms': 2,
        'Respondent used arms to stand up': 1,
        'Test not completed ': 0,
        "Don't know": np.nan,
        'Refusal': np.nan,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


df = map_chair_rise_results(df, ['chair_rise_result'])


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3684060542.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(mapping)


In [79]:
def map_walking_speed_results(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Maps walking speed results to numeric format.
    """
    df = df.copy()
    mapping = {
        "Completed successfully": 1,
        "Attempted but unable to complete": 0,
        "Stopped by the interviewer because of safety reasons": 0,
        "Not attempted, respondent felt it would": 0,
        "Respondent refused": np.nan,
        "Participant unable to understand instruction": np.nan,
        "Refusal": np.nan,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

df = map_walking_speed_results(df, ['first_walking_speed_result', 'second_walking_speed_result'])

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3945319162.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mapping)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3945319162.py:19: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(mapping)


In [80]:
def map_functional_numeric_codes(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Maps functional numeric codes to NaN or numeric values for specified columns.
    """
    df = df.copy()
    mapping = {
        '-15. no information': np.nan,
        '-13. not asked in this wave': np.nan,
        "-12. don't know / refusal": np.nan,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

df = map_functional_numeric_codes(df, ['adla', 'iadla', 'iadlza', 'lgmuscle', 'grossmotor', 'finemotor'])


In [81]:
def map_self_reported_health(df: pd.DataFrame, columns: List[str] = ['sphus']) -> pd.DataFrame:
    df = df.copy()
    mapping = {
        '-15. no information': np.nan,
        '-13. not asked in this wave': np.nan,
        "-12. don't know / refusal": np.nan,
        "Refusal": np.nan,
        "Don't know": np.nan,
        '1. Excellent': 5,
        '2. Very good': 4,
        '3. Good': 3,
        '4. Fair': 2,
        '5. Poor': 1,
        'Excellent': 5,
        'Very good': 4,
        'Good': 3,
        'Fair': 2,
        'Poor': 1,
        'SPONTANEOUS registered or legally blind': 1,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


df = map_self_reported_health(df, ['sphus', 'ph044_', 'ph046_', 'hearing', 'eyesight_close'])


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/607679537.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mapping)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/607679537.py:25: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(mapping)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/607679537.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly ca

In [82]:
def map_boolean_to_numeric(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    df = df.copy()
    mapping = {
        '-15. no information': np.nan,
        '-13. not asked in this wave': np.nan,
        "-12. don't know / refusal": np.nan,
        '-10. SHARELIFE interview': np.nan,
        "Refusal": np.nan,
        "Don't know": np.nan,
        '1. Yes': 1,
        'Yes': 1,
        '5. No': 0,
        'No': 0,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

df = map_boolean_to_numeric(df, ['ever_smoked', 'ph745_'])

In [83]:
def map_bmi_numeric(df: pd.DataFrame, columns: List[str] = ['bmi']) -> pd.DataFrame:
    df = df.copy()
    invalid_values = {
        '-15. no information': np.nan,
        '-13. not asked in this wave': np.nan,
        "-12. don't know / refusal": np.nan,
        '-3. implausible value/suspected wrong': np.nan,
    }

    for col in columns:
        if col not in df.columns:
            continue
        df[col] = df[col].replace(invalid_values)
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)

    return df

df = map_bmi_numeric(df)

In [84]:
def map_frequency_to_numeric(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:

    mapping = {
        'More than once a week': 1,
        'Once a week': 2,
        'One to three times a month': 3,
        'Hardly ever, or never': 4,
        "Don't know": np.nan,
        'Refusal': np.nan,
    }

    df = df.copy()
    for col in columns:
        if col in df.columns:
            df[col] = df[col].replace(mapping)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

df = map_frequency_to_numeric(df, ['br015_', 'br016_'])

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3542780056.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mapping)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3542780056.py:15: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(mapping)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/3542780056.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly

In [85]:
df

,mergeid,hhid2,mergeidp2,coupleid2,country,language,ph003_,ph004_,ph005_,ph006d1,...,dn043_,dn044_,isced1997_c1,isced1997_c2,isced1997_c3,isced1997_c4,isced1997_r,isced1997_sp,educational,educational_level
0,AT-000327-01,AT-000327-A,AT-000327-02,AT-000327-01-02,Austria,German (at),Poor,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
1,AT-000327-02,AT-000327-A,AT-000327-01,AT-000327-01-02,Austria,German (at),Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 2,ISCED-97 code 3,NaN,Medium,1.0
2,AT-001816-02,AT-001816-A,AT-001816-01,AT-001816-01-02,Austria,German (at),Good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 6,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
3,AT-002132-01,AT-002132-A,,,Austria,German (at),Poor,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 4,NaN,NaN,NaN,ISCED-97 code 2,ISCED-97 code 1,Low,0.0
4,AT-004234-01,AT-004234-A,AT-004234-02,AT-004234-01-02,Austria,German (at),Fair,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37138,SE-998278-01,SE-998278-A,SE-998278-02,SE-998278-01-02,Sweden,Swedish,Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,NaN,ISCED-97 code 2,NaN,Low,0.0
37139,SE-998807-01,SE-998807-A,SE-998807-02,SE-998807-01-02,Sweden,Swedish,Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 5,NaN,NaN,ISCED-97 code 5,NaN,High,2.0
37140,SE-998807-02,SE-998807-A,SE-998807-01,SE-998807-01-02,Sweden,Swedish,Good,No,"Limited, but not severely",0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 5,NaN,High,2.0
37141,SE-998962-01,SE-998962-A,,,Sweden,Swedish,Very good,No,Not limited,1.0,...,Yes,"No, marital status has not changed",ISCED-97 code 4,ISCED-97 code 4,NaN,NaN,ISCED-97 code 1,ISCED-97 code 3,Low,0.0


In [86]:
df.to_csv("data/main_df_after_merging.csv", index=False)

In [87]:
df = pd.read_csv("data/main_df_after_merging.csv")

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_69429/922147147.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,145,150,151,156,157,158,159,160,161,162,163,164,165,166,167,174,175,176,181,184,209,210,211,212,213,214,215,238,241,261,262,263,264,265,266,287,289,290,299,300) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/main_df_after_merging.csv")


In [88]:
df

,mergeid,hhid2,mergeidp2,coupleid2,country,language,ph003_,ph004_,ph005_,ph006d1,...,dn043_,dn044_,isced1997_c1,isced1997_c2,isced1997_c3,isced1997_c4,isced1997_r,isced1997_sp,educational,educational_level
0,AT-000327-01,AT-000327-A,AT-000327-02,AT-000327-01-02,Austria,German (at),Poor,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
1,AT-000327-02,AT-000327-A,AT-000327-01,AT-000327-01-02,Austria,German (at),Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 2,ISCED-97 code 3,NaN,Medium,1.0
2,AT-001816-02,AT-001816-A,AT-001816-01,AT-001816-01-02,Austria,German (at),Good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 6,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
3,AT-002132-01,AT-002132-A,NaN,NaN,Austria,German (at),Poor,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 4,NaN,NaN,NaN,ISCED-97 code 2,ISCED-97 code 1,Low,0.0
4,AT-004234-01,AT-004234-A,AT-004234-02,AT-004234-01-02,Austria,German (at),Fair,Yes,Severely limited,0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 3,NaN,Medium,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37138,SE-998278-01,SE-998278-A,SE-998278-02,SE-998278-01-02,Sweden,Swedish,Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 3,ISCED-97 code 3,ISCED-97 code 3,NaN,ISCED-97 code 2,NaN,Low,0.0
37139,SE-998807-01,SE-998807-A,SE-998807-02,SE-998807-01-02,Sweden,Swedish,Very good,No,Not limited,0.0,...,Yes,"No, marital status has not changed",ISCED-97 code 5,ISCED-97 code 5,NaN,NaN,ISCED-97 code 5,NaN,High,2.0
37140,SE-998807-02,SE-998807-A,SE-998807-01,SE-998807-01-02,Sweden,Swedish,Good,No,"Limited, but not severely",0.0,...,Yes,"No, marital status has not changed",NaN,NaN,NaN,NaN,ISCED-97 code 5,NaN,High,2.0
37141,SE-998962-01,SE-998962-A,NaN,NaN,Sweden,Swedish,Very good,No,Not limited,1.0,...,Yes,"No, marital status has not changed",ISCED-97 code 4,ISCED-97 code 4,NaN,NaN,ISCED-97 code 1,ISCED-97 code 3,Low,0.0
